In [41]:
# Import the dependencies.
import os
import numpy as np
import pandas as pd
import datetime as dt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func


from flask import Flask, jsonify
import nest_asyncio

In [42]:
# Database Setup
# Create engine using the `hawaii.sqlite` database file
engine = create_engine("sqlite:////Users/cillian/class/NU-VIRT-DATA-PT-02-2024-U-LOLC/02-Homework/10-Advanced-SQL/Starter_Code/Resources/hawaii.sqlite")
# Declare a Base using `automap_base()`
Base = automap_base()
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)
print(Base.classes.keys())

['measurement', 'station']


/var/folders/7h/qdlrm1j10hs1qd6zbbzwy8sr0000gn/T/ipykernel_58321/1880448189.py:7: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  Base.prepare(engine, reflect=True)


In [43]:
# Assign the measurement class to a variable called `Measurement` and
# the station class to a variable called `Station`
Measurement = Base.classes.measurement
Station = Base.classes.station

In [44]:
# Create a session
session = Session(engine)

In [45]:
# Flask setup
app = Flask(__name__)

@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start><br/>"
        f"/api/v1.0/<start>/<end><br/>"
    )

@app.route("/api/v1.0/precipitation")
def precipitation():
    #begin by creating a session
    session = Session(engine)

    #query precipitation data from the last year
    most_recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
    one_year_ago = (pd.to_datetime(most_recent_date) - pd.DateOffset(years=1)).strftime('%Y-%m-%d')
    results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= one_year_ago).all()

    session.close()

    #convert the precipitation results into a dictionary
    precipitation_dict = {date: prcp for date, prcp in results}
    return jsonify(precipitation_dict)

@app.route("/api/v1.0/stations")
def stations():
    session = Session(engine)

    #query all of the stations
    results = session.query(Station.station).all()

    session.close()

    #convert tuples into a normal list
    stations = list(np.ravel(results))
    return jsonify(stations)

@app.route("/api/v1.0/tobs")
def tobs():
    session = Session(engine)

    #query the dates and temp observations of the most active station from last year
    most_active_station = session.query(Measurement.station, func.count(Measurement.station))\
                                 .group_by(Measurement.station)\
                                 .order_by(func.count(Measurement.station).desc()).first()[0]
    one_year_ago = (pd.to_datetime(most_recent_date) - pd.DateOffset(years=1)).strftime('%Y-%m-%d')
    results = session.query(Measurement.date, Measurement.tobs)\
                     .filter(Measurement.station == most_active_station)\
                     .filter(Measurement.date >= one_year_ago).all()

    session.close()

    tobs = list(np.ravel(results))
    return jsonify(tobs)

@app.route("/api/v1.0/<start>")
def start_date(start):
    session = Session(engine)

    #query min, max, and avg temps from start date to end date, including the start date but not the end date 
    results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs))\
              .filter(Measurement.date >= start).all()

    session.close()

    temps = list(np.ravel(results))
    return jsonify(temps)

@app.route("/api/v1.0/<start>/<end>")
def start_end_date(start, end):
    session = Session(engine)

    #query min, max, and avg temps from start date to end date, including both the start and end date
    results = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs))\
              .filter(Measurement.date >= start).filter(Measurement.date <= end).all()

    session.close()

    temps = list(np.ravel(results))
    return jsonify(temps)

In [46]:
nest_asyncio.apply()

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Jul/2024 17:22:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jul/2024 17:22:33] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Jul/2024 17:23:50] "GET / HTTP/1.1" 200 -


In [ ]:
# Flash routes
/api/v1.0/precipitation
/api/v1.0/stations
/api/v1.0/tobs
/api/v1.0/
/api/v1.0//